# Face Recognition
### 1) Import libraries
### 2) Import video
### 3) Convert to b/w
### 4) Extract frames
### 5) Mark the region of face
### 6) Create Custom Dataset for each person
### 7) Create Train and test loaders
### 8) Create CNN Algorithm
### 9) Test and improve accuracy
### 9) Identify real time image

After we create the b/w images, then we will pass the images through an algorithm which will give the position of face in the image, then we will crop(resize) the image to that shape and we will fix a certain shape so as to pad these images and make every image of the same size. These images will be the input to Siamese Network.

Reference video to import files from gdrive into colab: https://www.youtube.com/watch?v=BuuH0wsJ8-k

In [1]:
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import OrderedDict
from tqdm import tqdm
import glob

import random
import matplotlib.pyplot as plt
from PIL import Image
from itertools import combinations 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms as transforms

## Just Replace this path with the path to your video directory
video_dir = "C:\\Users\\dhano\\Jupyter Notebooks\\Face Recognition\\Dataset\\Videos"

# if os.makedirs(path, exist_ok=True)
# Image_Dataset = pd.DataFrame(columns = ['label'] + [f'pixel_{i}' for i in range(600*600)])
# global Image_Dataset
# Triplet_Dataset = pd.DataFrame()
# Image_Dataset

In [2]:
person_name = OrderedDict()
person_name["Abhay Singh"] = 0
person_name["Abhishek Kumar"] = 1
person_name["Ajay Kumar"] = 2
person_name["Anand"] = 3
person_name["Ankit Soni"] = 4
person_name["Arjun Prajapat"] = 5
person_name["Arpit Jain"] = 6
person_name["Kushal Barochia"] = 7
person_name["Nakul Singh"] = 8
person_name["Ojas Lunia"] = 9
person_name["Pratik Wani"] = 10
person_name["Praveen"] = 11
person_name["Prince Kumar"] = 12
person_name["Raghav"] = 13
person_name["Rugved Kalaskar"] = 14
person_name["Shantanu"] = 15
person_name["Shrinath Mishra"] = 16
person_name["Siddharth Gupta"] = 17
person_name["Sohail"] = 18
person_name["Tejas Mathurkar"] = 19
person_name["Venkatesh"] = 20
person_name["Vikash"] = 21
person_name["Vivek"] = 22
person_name["Vrukshal Balki"] = 23
person_name["Yash Gupta"] = 24
OrderedDict()

OrderedDict()

##  Function to extract faces from video, crop them to same size and save them in a different folder

In [42]:
# Function to extract images from video, and save just the faces from all the images in a seperate folder
def video_to_faces(video_path, label,img_size):
    video = cv2.VideoCapture(video_path)
    if not video.isOpened():
        print(f"Error: Unable to open video file at {video_path}")
        return 0,0
    total_frames = video.get(cv2.CAP_PROP_FRAME_COUNT)
    # We can use os.chdir(path) to change the working directory
#     arr = np.array([])
    width, height =  video.get(cv2.CAP_PROP_FRAME_WIDTH), video.get(cv2.CAP_PROP_FRAME_HEIGHT)
    df = pd.DataFrame(columns = np.arange(int(width)*int(height)))
    face_classifier = cv2.CascadeClassifier(
        cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    max_h, max_w = 600,600
#     path = 'C:\\Users\\dhano\\Jupyter Notebooks\\Face Recognition\\' + str(label)
    path = video_path.replace('Videos', 'Images')
    path = path.replace('.mp4', '')
    os.makedirs(path, exist_ok=True)  # Ensure the directory exists or create it
    for i in range(0,int(total_frames),10):
        frame_id = i
        file_path = os.path.join(path , f'{frame_id}.jpg')
        if os.path.exists(file_path):
            continue
        else:
            video.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
            ret, frame = video.read()
            if not ret:
                print(f"Error reading frame {frame_id}")
                continue
            gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            face = face_classifier.detectMultiScale(
                gray_image, scaleFactor=1.2, minNeighbors=9, minSize=(250, 250), maxSize=(max_h  ,max_w))
#             x, y, w, h = 0,0,0,0
    #         print(face)
            if len(face) == 0: ## If no face is detected then it will skip this frame
                continue
            for (x, y, w, h) in face:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 4)
                x1 = abs(max_h//2 - x - w//2)
                x2 = abs(max_h//2 + x + w//2)
                y1 = abs(max_w//2 - y - h//2)
                y2 = abs(max_w//2 + y + h//2)
                im_crop = gray_image[y1:y2, x1:x2]
                im_crop = cv2.resize(im_crop, (img_size, img_size), interpolation=cv2.INTER_AREA) # try using 100*100
#                 flat_img = im_crop.flatten()
#                 print(flat_img)
#                 Modified the following code so that the image gets flattened and we will add the flattened image along with its label in. new rules. stop.
#                 new_row = pd.DataFrame([[label] + flat_img.tolist()])
#                 Image_Dataset = pd.concat([Image_Dataset, new_row], ignore_index=True)
                im_crop = cv2.cvtColor(im_crop, cv2.COLOR_BGR2RGB)
#                 im_crop = cv2.cvtColor(im_crop, cv2.COLOR_GRAY2RGB)
#             import matplotlib.pyplot as plt
#             plt.figure(figsize=(10,10))
#             plt.imshow(im_crop, cmap = 'gray')
#             plt.axis('off')
    #         path = 'C:\\Users\\dhano\\Jupyter Notebooks\\Face Recognition\\' + str(label)
#             print(im_crop.shape)
                cv2.imwrite(file_path, im_crop)
#     print("done")
    return 
# video_to_faces('Abhay Singh.mp4', "Abhay singh")
# video_to_faces('C:\\Users\\dhano\\Jupyter Notebooks\\Face Recognition\\Dataset\\Videos\\Abhay Singh.mp4', "Abhay Singh")
# print(max_h, max_w)
plt.close()

## Code to convert all the videos to face images and store them in seperate folders according to their names

In [43]:
## Code to be used later to get a list of sub-folders in a folder
## folders = [x[0] for x in os.walk(video_dir)]
img_size = 150
# Image_Dataset = pd.DataFrame(columns = ['label'] + [f'pixel_{i}' for i in range(img_size*img_size)])
videos_list = [f for f in os.listdir(video_dir) if f.endswith(('.mp4'))]
for video in tqdm(videos_list):
#     print("Working on", video) 
    video_path = video_dir + "\\" + str(video)
    label = video.replace('.mp4','')
    video_to_faces(video_path, label,img_size)
#     max_h, max_w = max(h, max_h), max(w, max_w)
#     print(max_h, max_w)
#     Image_Dataset.to_csv('Image_Dataset.csv')
print("Done")

 30%|████████████████████████▉                                                          | 9/30 [11:43<29:52, 85.36s/it]

Error reading frame 690


 63%|███████████████████████████████████████████████████▉                              | 19/30 [20:55<13:13, 72.14s/it]

Error reading frame 740


 83%|████████████████████████████████████████████████████████████████████▎             | 25/30 [27:32<05:57, 71.51s/it]

Error reading frame 350


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [28:58<00:00, 57.94s/it]

Done


## Generate Random Triplets

In [44]:
triplets_data = pd.DataFrame() 
## add all the created triplets to a Dataframe by flattening them by using df.loc[i] and save it to an excel file
def generate_random_triplets(images_path, pairs_per_class):
    '''
    We will choose the number of triplets based on the number of images in each folder.
    Lets say we will choose at max 8 pairs of anchor and positive images from a folder.
    If a folder has less than 5 images then we will skip it.
    '''
    triplets_path = images_path.replace("Images", "Triplets")
    os.makedirs(triplets_path, exist_ok=True)
    anchor = []
    pos = []
    neg = []
    folders = [x[0] for x in os.walk(images_path)]
    folders = folders[1:]
    for folder in tqdm(folders):
#         print(folders)
        path = os.path.join(images_path, folder)
        image_files = [f for f in os.listdir(path) if f.endswith('.jpg')]
#         print("Image_files", image_files)
        if not image_files:
            continue
        if len(image_files) < 4:
            #Skip folder with less than 8 images
            continue
        for i in range(pairs_per_class):
            current_path = os.path.join(triplets_path, str(f'{folders.index(folder)}_{i}'))
            os.makedirs(current_path, exist_ok=True)
            ## Choose anchor and positive images
            # Choose 2 random numbers to be the indexes for anchor and pos image
            anchor_idx, positive_idx = np.random.choice(len(image_files), size=2, replace=False)
            anchor_image = cv2.imread(os.path.join(path, image_files[anchor_idx]))
            pos_image = cv2.imread(os.path.join(path, image_files[positive_idx]))
            # For neg image we will randomly choose a folder and randomly choose an image from it
            neg_index = np.random.choice([x for x in range(0, len(folders)) if x != folder]) 
            neg_folder = os.path.join(images_path, folders[neg_index])
            neg_image_files = [f for f in os.listdir(neg_folder) if f.endswith(('.jpg'))]
            if not neg_image_files:
                continue
            neg_image = cv2.imread(os.path.join(neg_folder, np.random.choice(neg_image_files)))
            cv2.imwrite(os.path.join(current_path, "anchor.jpg"),anchor_image)
            cv2.imwrite(os.path.join(current_path, "pos.jpg"),pos_image)
            cv2.imwrite(os.path.join(current_path, "neg.jpg"),neg_image)
            anchor.append(anchor_image)
            pos.append(pos_image)
            neg.append(neg_image)
    return anchor, pos, neg
image_dir = video_dir.replace("Videos", "Images")
anchor, pos, neg = generate_random_triplets(image_dir,8)

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:05<00:00,  6.00it/s]


## Custom Dataset Class for the triplets

In [3]:
class Dataset(Dataset):
    def __init__(self, triplet_folder, transform): #transforms.ToTensor()
        self.transform = transform
        folders = [x[0] for x in os.walk(triplet_folder)]
        self.folders = folders[1:]        
        
    def __len__(self):
        return len(self.folders)
    
    def __getitem__(self, idx):
        folder = self.folders[idx]
        label = str(folder)
        label = folder.split("\\")[-1]
        label = label.split("_")[0]
        
        self.label = label
        #Edit the current path
        current_path = os.path.join(triplet_folder, str(folder))
        image_files = [f for f in os.listdir(current_path) if f.endswith('.jpg')]
#         print("Folder: ",folder)
        anchor = (cv2.imread(os.path.join(current_path, "anchor.jpg"))).astype(np.float32)
        pos = cv2.imread(os.path.join(current_path, "pos.jpg")).astype(np.float32)
        neg = cv2.imread(os.path.join(current_path, "neg.jpg")).astype(np.float32)
#         print(f"anchor: {anchor}\npos: {pos}\nneg: {neg}")
        self.anchor = self.transform(anchor).view(3,150,150) # tensor -> from_numpy
        self.pos = self.transform(pos).view(3,150,150)
        self.neg = self.transform(neg).view(3,150,150)
        return self.label, self.anchor, self.pos, self.neg
        
triplet_folder = video_dir.replace("Videos", "Triplets")
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
Triplet_Dataset = Dataset(triplet_folder, transform=transform)

## Split Dataset into train and test

In [4]:
train_size = int(0.9 * len(Triplet_Dataset))
test_size = len(Triplet_Dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(Triplet_Dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=12)
test_loader = DataLoader(test_dataset)
examples = enumerate(test_loader)
i, (l,x,y,z) = next(examples)
l[0]
x.shape

torch.Size([1, 3, 150, 150])

## CNN Code

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, models
from PIL import Image
import os

# Define a Siamese network with VGG-16 as the base network
class SiameseNetwork(nn.Module):
    def __init__(self, vgg16):
        super(SiameseNetwork, self).__init__()
        self.vgg16 = vgg16
        self.fc = nn.Linear(1000, 128)  # Assuming VGG-16 output size is 1000, adjust if needed

    def forward_one(self, x):
        x = self.vgg16(x)
        x = x.view(x.size()[0], -1)
        x = self.fc(x)
        return x

    def forward(self, anchor, positive, negative):
#         anchor = anchor.to(torch.ByteTensor)
#         pos = pos.to(torch.ByteTensor)
#         neg = neg.to(torch.ByteTensor)
        output1 = self.forward_one(anchor)
        output2 = self.forward_one(positive)
        output3 = self.forward_one(negative)
        return output1, output2, output3

# Custom dataset for loading triplets
# Triplet loss function
class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative):
        distance_positive = torch.norm(anchor - positive, dim=1)
        distance_negative = torch.norm(anchor - negative, dim=1)
        losses = nn.functional.relu(distance_positive - distance_negative + self.margin)
        return losses.mean()

# Load VGG-16 pre-trained on ImageNet
vgg16 = models.vgg16(pretrained=False)
vgg16.features = nn.Sequential(*list(vgg16.features.children())[:-1])  # Remove the last fully connected layer

# Create Siamese network
siamese_net = SiameseNetwork(vgg16)


## Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = 'cpu'
siamese_net.to(device)


C:\Users\dhano\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
C:\Users\dhano\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


SiameseNetwork(
  (vgg16): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
     

In [6]:
if os.path.exists("C:\\Users\\dhano\\Jupyter Notebooks\\Face Recognition\\31Oct.pth"):
    siamese_net.load_state_dict(torch.load("C:\\Users\\dhano\\Jupyter Notebooks\\Face Recognition\\31Oct.pth"))
    siamese_net.eval()

In [ ]:
triplet_loss = TripletLoss()

# Set up the optimizer
optimizer = optim.Adam(siamese_net.parameters(), lr=0.001)

# Load your triplets data into the TripletDataset class
# You may need to customize the paths and transformations based on your dataset

# Training loop
num_epochs = 10
siamese_net.train()
for epoch in range(num_epochs):
    for batch in tqdm(train_loader):
        _, anchor, pos, neg = batch
        anchor, pos, neg = anchor.to(device), pos.to(device), neg.to(device)
        optimizer.zero_grad()
        output1, output2, output3 = siamese_net(anchor, pos, neg)
        loss = triplet_loss(output1, output2, output3)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [18]:
import cv2
import torch
from torchvision import transforms

identification_dir = video_dir.replace("Videos", "Identification")
encodings = []
representatives = [img for img in os.listdir(identification_dir) if img.endswith('.jpg')]
siamese_net.eval()

# Define normalization transform
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

with torch.no_grad():
    for img in representatives:
        image = cv2.imread(os.path.join(identification_dir, img)).astype(np.float32)
#         print(image.shape)
        image = transform(image)
        image = image.unsqueeze(0).to(device)  # Add batch dimension
        encoding = siamese_net.forward_one(image)
        encodings.append(encoding)

# Convert the list of encodings to a tensor
encodings_tensor = torch.cat(encodings, dim=0)

print(encodings_tensor.shape)


torch.Size([25, 128])


## Use the array of encodings created in the earlier Block of code. to calculate the distances between the encoding of a test image and the encoding of the representatives of all the classes. The minimum distance will give the predicted class of the image. 

In [19]:
if __name__ == "__main__":
    def identify(image, encodings, label):
        # Read and normalize the image
    #     image = cv2.imread(image_path).astype(np.float32)
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
        # Forward pass to obtain the encoding
        with torch.no_grad():
            encoding = siamese_net.forward_one(image)

        # Calculate distances
        distances = torch.norm(encodings - encoding, dim=1).to('cpu')
        distances = distances.numpy()
    #     print("Distances:", distances)
        min_distance_index = np.argmin(distances)
        print("Identified as:", min_distance_index, "Originally: ", label)
        return min_distance_index

    # Example usage self.anchor = self.transform(anchor).view(3,150,150)
    # path = "C:\\Users\\dhano\\Jupyter Notebooks\\Face Recognition\\Dataset\\Images\\Venkatesh\\320.jpg"
    count = 0
    total = len(test_loader)
    for batch in test_loader:
        label, anchor, pos, neg = batch
    #     anchor = anchor.squeeze().numpy().astype(np.float32)
        anchor = anchor.to(device)
        print(anchor.shape)
        index = identify(anchor, encodings_tensor, label)
        if index == int(label[0]):
            count = count + 1
    print(f"{count}/{total}")
        


torch.Size([1, 3, 150, 150])
Identified as: 12 Originally:  ('5',)
torch.Size([1, 3, 150, 150])
Identified as: 15 Originally:  ('0',)
torch.Size([1, 3, 150, 150])
Identified as: 10 Originally:  ('4',)
torch.Size([1, 3, 150, 150])
Identified as: 0 Originally:  ('0',)
torch.Size([1, 3, 150, 150])
Identified as: 3 Originally:  ('6',)
torch.Size([1, 3, 150, 150])
Identified as: 10 Originally:  ('19',)
torch.Size([1, 3, 150, 150])
Identified as: 10 Originally:  ('1',)
torch.Size([1, 3, 150, 150])
Identified as: 19 Originally:  ('17',)
torch.Size([1, 3, 150, 150])
Identified as: 23 Originally:  ('11',)
torch.Size([1, 3, 150, 150])
Identified as: 23 Originally:  ('19',)
torch.Size([1, 3, 150, 150])
Identified as: 14 Originally:  ('11',)
torch.Size([1, 3, 150, 150])
Identified as: 8 Originally:  ('16',)
torch.Size([1, 3, 150, 150])
Identified as: 10 Originally:  ('2',)
torch.Size([1, 3, 150, 150])
Identified as: 1 Originally:  ('1',)
torch.Size([1, 3, 150, 150])
Identified as: 0 Originally:  (

# Code to Open Webcam, draw a rectangle around face and label it


In [196]:
# access video from the webcam
video_capture = cv2.VideoCapture(0)
face_classifier = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
def detect_bounding_box(vid):
    gray_image = cv2.cvtColor(vid, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray_image, 1.1, 5, minSize=(40, 40))
    x, y, w, h = 0,0,0,0
    for (x, y, w, h) in faces:
        cv2.rectangle(vid, (x, y), (x + w, y + h), (0, 255, 0), 4)
        x, y, w, h = x, y, w, h
    return faces, x, y
while True:
    result, video_frame = video_capture.read()  # read frames from the video
    if result is False:
        break  # terminate the loop if the frame is not read successfully
    faces, x, y = detect_bounding_box(
        video_frame
    )  # apply the function we created to the video frame
    title = 'Face'
    cv2.putText(video_frame,title , (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (30,255,30), 2)
    cv2.imshow(
        "My Face Detection Project", video_frame
    )
    # display the processed frame in a window named "My Face Detection Project"
    # add label to the face
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
video_capture.release()
cv2.destroyAllWindows()